<a href="https://colab.research.google.com/github/NavdeepTU/Deep-Learning/blob/main/02.%20ML%20basics%20with%20Keras/08.%20Tune%20hyperparameters%20with%20keras%20tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
# install and import the keras tuner
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.2 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt

In [4]:
# download and prepare the dataset
(img_train, label_train), (img_test, label_test) = tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
# normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [6]:
# Define the model

def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape = (28, 28)))

  # tune the number of units in the first Dense layer
  # choose an optimal value between 32 - 512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(tf.keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(tf.keras.layers.Dense(10))

  # tune the learning rate for the optimizer
  # choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(
      optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
      metrics = ['accuracy']
  )

  return model

In [7]:
# instantiate the tuner and perform hypertuning

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [8]:
# create a callback to stop training early after reaching a certain value for
# the validation loss

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [9]:
# run the hyperparameter search

tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f'\
The hyperparameter search is complete. The optimal number of units in the first densely-connected \
layer is {best_hps.get("units")} and the optimal learning rate for the optimizer is \
{best_hps.get("learning_rate")}.')

Trial 30 Complete [00h 01m 23s]
val_accuracy: 0.8663333058357239

Best val_accuracy So Far: 0.8918333053588867
Total elapsed time: 00h 19m 19s
The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is 128 and the optimal learning rate for the optimizer is 0.001.


In [10]:
# train the model
# build the model with optimal hyperparameters and train it on the data for 50 epochs

model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch, ))

Epoch 1/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5173 - accuracy: 0.8186 - val_loss: 0.4130 - val_accuracy: 0.8533
Epoch 2/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3892 - accuracy: 0.8606 - val_loss: 0.3701 - val_accuracy: 0.8692
Epoch 3/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3472 - accuracy: 0.8739 - val_loss: 0.3516 - val_accuracy: 0.8714
Epoch 4/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3200 - accuracy: 0.8830 - val_loss: 0.3400 - val_accuracy: 0.8770
Epoch 5/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3034 - accuracy: 0.8882 - val_loss: 0.3412 - val_accuracy: 0.8762
Epoch 6/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2869 - accuracy: 0.8937 - val_loss: 0.3296 - val_accuracy: 0.8809
Epoch 7/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2746 - accuracy: 0.8976 - val_loss: 0.3271 - val_accuracy:

In [11]:
# re-instantiate the hypermodel and train it with the optimal number of epochs from above.

hypermodel = tuner.hypermodel.build(best_hps)

# retrain the model
hypermodel.fit(img_train, label_train, epochs = best_epoch, validation_split=0.2)

Epoch 1/27
1500/1500 [==============================] - 9s 5ms/step - loss: 0.5208 - accuracy: 0.8158 - val_loss: 0.4311 - val_accuracy: 0.8478
Epoch 2/27
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3898 - accuracy: 0.8607 - val_loss: 0.4050 - val_accuracy: 0.8528
Epoch 3/27
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3475 - accuracy: 0.8741 - val_loss: 0.3428 - val_accuracy: 0.8742
Epoch 4/27
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3211 - accuracy: 0.8828 - val_loss: 0.3502 - val_accuracy: 0.8723
Epoch 5/27
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3026 - accuracy: 0.8892 - val_loss: 0.3508 - val_accuracy: 0.8716
Epoch 6/27
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2873 - accuracy: 0.8952 - val_loss: 0.3317 - val_accuracy: 0.8805
Epoch 7/27
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2738 - accuracy: 0.8980 - val_loss: 0.3252 - val_accuracy:

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print('[test loss, test accuracy]:', eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.3980 - accuracy: 0.8875
[test loss, test accuracy]: [0.3980485200881958, 0.887499988079071]
